# **Deep Learning Course**

## **Loss Functions and Multilayer Perceptrons (MLP)**

---

### **Student Information:**

- **Name:** *Alireza Noori*
- **Student Number:** *402210021*

---

### **Assignment Overview**

In this notebook, we will explore various loss functions used in neural networks, with a specific focus on their role in training **Multilayer Perceptrons (MLPs)**. By the end of this notebook, you will have a deeper understanding of:
- Types of loss functions
- How loss functions affect the training process
- The relationship between loss functions and model optimization in MLPs

---

### **Table of Contents**

1. Introduction to Loss Functions
2. Types of Loss Functions
3. Multilayer Perceptrons (MLP)
4. Implementing Loss Functions in MLP
5. Conclusion

---



# 1.Introduction to Loss Functions

In deep learning, **loss functions** play a crucial role in training models by quantifying the difference between the predicted outputs and the actual targets. Selecting the appropriate loss function is essential for the success of your model. In this assay, we will explore various loss functions available in PyTorch, understand their theoretical backgrounds, and provide you with a scaffolded class to experiment with these loss functions.

Before begining, let's train a simle MLP model using the **L1Loss** function. We'll return to this model later to experiment with different loss functions. We'll start by importing the necessary libraries and defining the model architecture.

First things first, let's talk about **L1Loss**.

### 1. L1Loss (`torch.nn.L1Loss`)
- **Description:** Also known as Mean Absolute Error (MAE), L1Loss computes the average absolute difference between the predicted values and the target values.
- **Use Case:** Suitable for regression tasks where robustness to outliers is desired.

Here is the mathematical formulation of L1Loss:
\begin{equation}
\text{L1Loss} = \frac{1}{n} \sum_{i=1}^{n} |y_{\text{pred}_i} - y_{\text{true}_i}|
\end{equation}

Let's implement a simple MLP model using the L1Loss function.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from torch.optim import Adam
# Don't be courious about Adam, it's just a fancy name for a fancy optimization algorithm

Here, we'll define a class called `SimpleMLP` that inherits from `nn.Module`. This class can have multiple layers, and we'll use the `nn.Sequential` module to define the layers of the model. The model will have the following architecture:

In [2]:
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_hidden_layers=1, last_layer_activation_fn=nn.ReLU):
        super(SimpleMLP, self).__init__()
        # TODO: Define the layers of the MLP

        layers = [] # a list for holding layers

        layers.append(nn.Linear(input_dim, hidden_dim)) # input layer to first hidden layer
        layers.append(nn.ReLU())

        for _ in range(num_hidden_layers - 1): # hidden layers
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.ReLU())

        layers.append(nn.Linear(hidden_dim, output_dim)) # output layer

        if last_layer_activation_fn:
            layers.append(last_layer_activation_fn()) # final activation function

        # define the model
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        # TODO: Define the forward pass of the MLP
        fowrard_pass = self.model(x)
        return fowrard_pass

Now, let's define a class called `SimpleMLP_Loss` that has the following architecture:

In [3]:
from tqdm import tqdm

class SimpleMLPTrainer:
    def __init__(self, model, criterion, optimizer):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer

    def train(self, train_loader, num_epochs):
        train_losses = []
        #TODO: Implement the training loop
        #Note: You should also print the training loss at each epoch, use tqdm for progress bar
        #Note: You should return the training loss at each epoch
        for epoch in range(num_epochs):
            self.model.train()
            running_loss = 0.0

            # Iterate over training data in batches
            for inputs, targets in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch'):
                self.optimizer.zero_grad() # zero the parameter gradients
                # forward
                outputs = self.model(inputs)
                loss = self.criterion(outputs, targets)

                # backward and optimization
                loss.backward()
                self.optimizer.step()

                # summing up loss
                running_loss += loss.item()

            # average loss for this epoch
            avg_loss = running_loss / len(train_loader)
            train_losses.append(avg_loss)
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

        return train_losses

    def evaluate(self, val_loader):
        #TODO: Implement the evaluation loop
        #Note: You should return the validation loss and accuracy
        self.model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, targets in val_loader:
                # forward
                outputs = self.model(inputs)
                loss = self.criterion(outputs, targets)
                val_loss += loss.item()

                # get predictions by binary classification
                predicted = (outputs > 0.5).float()

                total += targets.size(0)
                correct += (predicted == targets).sum().item()

        # average validation loss and accuracy
        avg_val_loss = val_loss / len(val_loader)
        accuracy = (correct / total) * 100  # Calculate accuracy as a percentage

        return avg_val_loss, accuracy

Next, lets test our model using the L1Loss function. You'll use <span style="color:red">*Titanic Dataset*</span> to train the model.


In [4]:
from torch.nn import L1Loss
# Load dataset
train_url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
data = pd.read_csv(train_url)

# Preprocessing (simple example)
data = data[['Pclass', 'Sex', 'Age', 'Fare', 'Survived']].dropna()
data['Sex'] = data['Sex'].map({'male': 0, 'female': 1})

# TODO: Convert the data to PyTorch tensors and create a DataLoader
X = data[['Pclass', 'Sex', 'Age', 'Fare']].values
y = data['Survived'].values

# TODO: Split the data into training and validation sets
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)  # reshape y to be a vector
dataset = TensorDataset(X_tensor, y_tensor)
train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=42) #(80% training, 20% validation)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)


<div style="text-align: center;"> <span style="color:red; font-size: 26px; font-weight: bold;">Let's train!</span> </div>

In [5]:
from torch.nn import L1Loss

# TODO: Define the model, criterion, and optimizer
input_dim = X.shape[1]
hidden_dim = 64
output_dim = 1
model = SimpleMLP(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_hidden_layers=2)
criterion = L1Loss()
optimizer = Adam(model.parameters(), lr=0.001)

# TODO: Train the model
trainer = SimpleMLPTrainer(model=model, criterion=criterion, optimizer=optimizer)
num_epochs = 10
train_losses = trainer.train(train_loader, num_epochs)

# TODO: Evaluate the model
val_loss, val_accuracy = trainer.evaluate(val_loader)
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

Epoch 1/10: 100%|██████████| 18/18 [00:00<00:00, 357.44batch/s]


Epoch [1/10], Loss: 0.4104


Epoch 2/10: 100%|██████████| 18/18 [00:00<00:00, 660.23batch/s]


Epoch [2/10], Loss: 0.4091


Epoch 3/10: 100%|██████████| 18/18 [00:00<00:00, 701.02batch/s]


...

Epoch [8/10], Loss: 0.4088


Epoch 9/10: 100%|██████████| 18/18 [00:00<00:00, 607.13batch/s]


Epoch [9/10], Loss: 0.4101


Epoch 10/10: 100%|██████████| 18/18 [00:00<00:00, 635.94batch/s]


Epoch [10/10], Loss: 0.4101
Validation Loss: 0.3925, Validation Accuracy: 60.84%


---
# 2. Types of Loss Functions

PyTorch offers a variety of built-in loss functions tailored for different types of problems, such as regression, classification, and more. Below, we discuss several commonly used loss functions, their theoretical foundations, and typical use cases.

### 2. MSELoss (`torch.nn.MSELoss`)
- **Description:** Mean Squared Error (MSE) calculates the average of the squares of the differences between predicted and target values.
- **Use Case:** Commonly used in regression problems where larger errors are significantly penalized.

Here is boring math stuff for MSE:
\begin{equation}
\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_{i} - \hat{y}_{i})^{2}
\end{equation}

<span style="color:red; font-size: 18px; font-weight: bold;">Warning:</span> Don't forget to reinitialize the model before experimenting with different loss functions.

In [6]:
from torch.nn import MSELoss

# Define the model, criterion, and optimizer
input_dim = X.shape[1]
hidden_dim = 64
output_dim = 1
model = SimpleMLP(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_hidden_layers=2)
criterion = MSELoss()
optimizer = Adam(model.parameters(), lr=0.001)

# TODO: Train the model
trainer = SimpleMLPTrainer(model=model, criterion=criterion, optimizer=optimizer)
train_losses = trainer.train(train_loader, num_epochs=10)

# TODO: Evaluate the model
val_loss, val_accuracy = trainer.evaluate(val_loader)
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")


Epoch 1/10: 100%|██████████| 18/18 [00:00<00:00, 587.30batch/s]


Epoch [1/10], Loss: 1.3206


Epoch 2/10: 100%|██████████| 18/18 [00:00<00:00, 640.06batch/s]


Epoch [2/10], Loss: 0.4101


Epoch 3/10: 100%|██████████| 18/18 [00:00<00:00, 590.77batch/s]


...

Epoch [8/10], Loss: 0.4085


Epoch 9/10: 100%|██████████| 18/18 [00:00<00:00, 633.74batch/s]


Epoch [9/10], Loss: 0.4108


Epoch 10/10: 100%|██████████| 18/18 [00:00<00:00, 478.42batch/s]

Epoch [10/10], Loss: 0.4098
Validation Loss: 0.3925, Validation Accuracy: 60.84%


### 3. NLLLoss (`torch.nn.NLLLoss`)
- **Description:** Negative Log-Likelihood Loss measures the likelihood of the target class under the predicted probability distribution.
- **Use Case:** Typically used in multi-class classification tasks, especially when combined with `log_softmax` activation.

Here is the mathematical formulation of NLLLoss:
\begin{equation}
\text{NLLLoss} = -\frac{1}{n} \sum_{i=1}^{n} \log(y_{i})
\end{equation}

I hope you note the logarithm in the formula. It's important!

Why?

In this part, run your training with Relu at last layer. <span style="color:red; font-weight: bold;">Discuss </span> and explain the difference between the results of the two models. Find a proper solution to the problem.


In [7]:
# Run with relu activation function
from torch.nn import NLLLoss

# TODO: Train the model

# TODO: Evaluate the model

In [8]:
# Run with --- activation function
from torch.nn import NLLLoss

# TODO: Train the model

# TODO: Evaluate the model

Your reason for your choice:

<div>
**Your answer here**
</div>


### 4. CrossEntropyLoss (`torch.nn.CrossEntropyLoss`)
- **Description:** Combines `LogSoftmax` and `NLLLoss` in one single class. It computes the cross-entropy loss between the target and the output logits.
- **Use Case:** Widely used for multi-class classification problems.

The mathematical formulation of CrossEntropyLoss is as follows:
\begin{equation}
  \text{CrossEntropy}(y, \hat{y}) = - \sum_{i=1}^{C} y_i \log\left(\frac{e^{\hat{y}_i}}{\sum_{j=1}^{C} e^{\hat{y}_j}}\right)
\end{equation}
  where:
  - \( C \) is the number of classes,
  - \( y_i \) is a one-hot encoded target vector (or a scalar class label),
  - \( \hat{y}_i \) represents the logits (unnormalized model outputs) for each class.
  
  In practice, `torch.nn.CrossEntropyLoss` expects raw logits as input and internally applies the softmax function to convert the logits into probabilities, followed by the negative log-likelihood computation.

- **Background:** Cross-entropy measures the difference between the true distribution \( y \) and the predicted distribution \( \hat{y} \). The function minimizes the negative log-probability assigned to the correct class, effectively penalizing predictions that deviate from the true class, making it a standard choice for classification tasks in deep learning.

Now, let's implement a class called `SimpleMLP_Loss` that has the following architecture:


In [9]:
from torch.nn import CrossEntropyLoss


# Define the model, criterion, and optimizer
input_dim = X.shape[1]
hidden_dim = 64
output_dim = 1
model = SimpleMLP(input_dim, hidden_dim, output_dim, num_hidden_layers=1, last_layer_activation_fn=nn.ReLU)
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

#TODO Train the model
trainer = SimpleMLPTrainer(model=model, criterion=criterion, optimizer=optimizer)
num_epochs = 10
train_losses = trainer.train(train_loader, num_epochs)

#TODO Evaluate the model
val_loss, val_accuracy = trainer.evaluate(val_loader)
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")


Epoch 1/10: 100%|██████████| 18/18 [00:00<00:00, 479.31batch/s]


Epoch [1/10], Loss: 0.0000


Epoch 2/10: 100%|██████████| 18/18 [00:00<00:00, 686.19batch/s]


Epoch [2/10], Loss: 0.0000


Epoch 3/10: 100%|██████████| 18/18 [00:00<00:00, 680.68batch/s]


...

Epoch [8/10], Loss: 0.0000


Epoch 9/10: 100%|██████████| 18/18 [00:00<00:00, 568.92batch/s]


Epoch [9/10], Loss: 0.0000


Epoch 10/10: 100%|██████████| 18/18 [00:00<00:00, 656.71batch/s]

Epoch [10/10], Loss: 0.0000
Validation Loss: 0.0000, Validation Accuracy: 60.84%



### 5. KLDivLoss (`torch.nn.KLDivLoss`)
- **Description:** Kullback-Leibler Divergence Loss measures how one probability distribution diverges from a second, reference distribution. Unlike other loss functions that focus on classification, KL divergence specifically compares the relative entropy between two distributions. It quantifies the information loss when using the predicted distribution to approximate the true distribution.

- **Mathematical Function:**
\begin{equation}
  \text{KL}(P \parallel Q) = \sum_{i=1}^{C} P(i) \left( \log P(i) - \log Q(i) \right)
\end{equation}
  where:
  - \( P \) is the target (true) probability distribution,
  - \( Q \) is the predicted distribution (often the output of `log_softmax`),
  - \( C \) is the number of classes.

  KL divergence is always non-negative, and it equals zero if the two distributions are identical. The loss function expects the model's output to be in the form of log-probabilities (using `log_softmax`) and compares this against a target probability distribution, which is typically a normalized distribution (using softmax).

- **Use Case:** KLDivLoss is frequently used in:
  - **Variational Autoencoders (VAEs):** In VAEs, KL divergence is used to measure how much the learned latent space distribution deviates from a prior distribution (often Gaussian).
  - **Knowledge Distillation:** In teacher-student models, KL divergence is used to transfer the "soft" knowledge from a teacher model to a student model by comparing their output probability distributions.
  - **Reinforcement Learning:** It can be used to update policies while minimizing the divergence from a previous policy.

- **Background:** Kullback-Leibler divergence, a core concept in information theory, measures the inefficiency of assuming the predicted distribution \( Q \) when the true distribution is \( P \). It is asymmetric, meaning that \( KL(P \parallel Q) \neq KL(Q \parallel P) \), so the direction of the comparison matters.

Again, in this part, run your training with Relu at last layer. <span style="color:red; font-weight: bold;">Discuss </span> and explain the difference between the results of the two models. Find a proper solution to the problem.


In [13]:
# Run with relu activation function
from torch.nn import KLDivLoss, ReLU

input_dim = X.shape[1]
hidden_dim = 64
output_dim = 1
model = SimpleMLP(input_dim, hidden_dim, output_dim, num_hidden_layers=1, last_layer_activation_fn=nn.ReLU)
criterion = KLDivLoss(reduction='batchmean')
optimizer = Adam(model.parameters(), lr=0.001)

# TODO: Train the model
trainer = SimpleMLPTrainer(model=model, criterion=criterion, optimizer=optimizer)
num_epochs = 10  # You can adjust this number based on your needs
train_losses = trainer.train(train_loader, num_epochs)

# TODO: Evaluate the model
val_loss, val_accuracy = trainer.evaluate(val_loader)
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")


Epoch 1/10: 100%|██████████| 18/18 [00:00<00:00, 765.21batch/s]


Epoch [1/10], Loss: 0.0000


Epoch 2/10: 100%|██████████| 18/18 [00:00<00:00, 835.70batch/s]


Epoch [2/10], Loss: 0.0000


Epoch 3/10: 100%|██████████| 18/18 [00:00<00:00, 859.88batch/s]


...

Epoch [8/10], Loss: 0.0000


Epoch 9/10: 100%|██████████| 18/18 [00:00<00:00, 776.20batch/s]


Epoch [9/10], Loss: 0.0000


Epoch 10/10: 100%|██████████| 18/18 [00:00<00:00, 725.55batch/s]

Epoch [10/10], Loss: 0.0000
Validation Loss: 0.0000, Validation Accuracy: 60.84%


In [14]:
# Run with relu activation function
from torch.nn import KLDivLoss

input_dim = X.shape[1]
hidden_dim = 64
output_dim = 1
model = SimpleMLP(input_dim, hidden_dim, output_dim, num_hidden_layers=1, last_layer_activation_fn=nn.LogSoftmax)
criterion = KLDivLoss(reduction='batchmean')
optimizer = Adam(model.parameters(), lr=0.001)

# TODO: Train the model
trainer = SimpleMLPTrainer(model=model, criterion=criterion, optimizer=optimizer)
num_epochs = 10  # You can adjust this number based on your needs
train_losses = trainer.train(train_loader, num_epochs)

# TODO: Evaluate the model
val_loss, val_accuracy = trainer.evaluate(val_loader)
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

Epoch 1/10:   0%|          | 0/18 [00:00<?, ?batch/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1553: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
Epoch 1/10: 100%|██████████| 18/18 [00:00<00:00, 352.78batch/s]


Epoch [1/10], Loss: 0.0000


Epoch 2/10: 100%|██████████| 18/18 [00:00<00:00, 819.93batch/s]


Epoch [2/10], Loss: 0.0000


Epoch 3/10: 100%|██████████| 18/18 [00:00<00:00, 761.63batch/s]


...

Epoch [8/10], Loss: 0.0000


Epoch 9/10: 100%|██████████| 18/18 [00:00<00:00, 714.35batch/s]


Epoch [9/10], Loss: 0.0000


Epoch 10/10: 100%|██████████| 18/18 [00:00<00:00, 753.30batch/s]

Epoch [10/10], Loss: 0.0000
Validation Loss: 0.0000, Validation Accuracy: 60.84%


Your reason for your choice:

<div>
**KLDivLoss requires the output of the model to be log-probabilities, and typically, the model uses a LogSoftmax activation in the final layer when using this loss function. ReLU would not be suitable in this case, as it is typically used for non-negative outputs, and would not give log-probabilities.**
</div>

### 6. CosineEmbeddingLoss (`torch.nn.CosineEmbeddingLoss`)
- **Description:** Measures the cosine similarity between two input tensors, `x1` and `x2`, and computes the loss based on a label `y` that indicates whether the tensors should be similar (`y = 1`) or dissimilar (`y = -1`). Cosine similarity focuses on the angle between vectors, disregarding their magnitude.

- **Mathematical Function:**
\begin{equation}
  \text{CosineEmbeddingLoss}(x1, x2, y) =
  \begin{cases}
  1 - \cos(x_1, x_2), & \text{if } y = 1 \\
  \max(0, \cos(x_1, x_2) - \text{margin}), & \text{if } y = -1
  \end{cases}
\end{equation}
  where $ \cos(x_1, x_2) $ is the cosine similarity between the two vectors, and `margin` is a threshold that determines how dissimilar the vectors should be.

- **Use Case:** Commonly used in tasks like face verification, image similarity, and other scenarios where the relative orientation of vectors (angle) is more important than their length, such as in embeddings and metric learning.

- **Background:** Cosine similarity compares the directional alignment of vectors, making it ideal for high-dimensional data where the magnitude may not be as informative. This loss is particularly useful when training models to learn meaningful embeddings that capture semantic similarity.

You'll become more fimiliar with this loss function in future.

---

# Regularization in Machine Learning

## Introduction

Regularization is a fundamental technique in machine learning that helps prevent overfitting by adding a penalty to the loss function. This penalty discourages the model from becoming too complex, ensuring better generalization to unseen data. In this notebook, you will explore the concepts of regularization, understand different types of regularization techniques, and apply them using Python's popular libraries.

## What is Regularization?

Regularization involves adding a regularization term to the loss function used to train machine learning models. This term imposes a constraint on the model's coefficients, effectively reducing their magnitude. By doing so, regularization helps in:

- **Preventing Overfitting:** Ensures the model does not become too tailored to the training data.
- **Improving Generalization:** Enhances the model's performance on new, unseen data.
- **Feature Selection:** Especially in L1 regularization, it can drive some coefficients to zero, effectively selecting important features.

## Types of Regularization

There are several types of regularization techniques, each imposing different constraints on the model's parameters:

### 1. L1 Regularization (Lasso)

L1 regularization adds the absolute value of the magnitude of coefficients as a penalty term to the loss function. It can lead to sparse models where some feature coefficients are exactly zero.

### 2. L2 Regularization (Ridge)

L2 regularization adds the squared magnitude of coefficients as a penalty term to the loss function. It tends to shrink the coefficients evenly but does not set them to zero.

### 3. Elastic Net

Elastic Net combines both L1 and L2 regularization penalties. It balances the benefits of both Lasso and Ridge methods, allowing for feature selection and coefficient shrinkage.

## Homework Time!
Import Iris dataset from sklearn.datasets and apply ridge regression with different alpha values. Then, create a gif that shows the changes of the classification boundary with respect to alpha values.

Import the libs that you need and start coding!

In [15]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from PIL import Image
from io import BytesIO
import imageio
import warnings


# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

Load the Iris dataset and select Setosa and Versicolor classes

In [18]:
# 1. Load and Prepare the Iris Dataset
iris = load_iris()

# Select only two classes for binary classification (Setosa and Versicolor)
X = iris.data[iris.target != 2, :2]
y = iris.target[iris.target != 2]

# split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


Define Function to Plot Decision Boundary

In [19]:
def plot_decision_boundary(model, X, y, alpha):
    # Define the grid (use meshgrid)
    h = 0.02  # Step size in the meshgrid
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    # Predict over the grid
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # Create a figure
    fig, ax = plt.subplots(figsize=(6, 5))

    # Plot the decision boundary
    ax.contourf(xx, yy, Z, alpha=0.3, levels=[-0.1, 0.1, 1.1], colors=['blue', 'red'])

    # Scatter plot of the training data
    scatter = ax.scatter(
        X[:, 0], X[:, 1], c=y, cmap='bwr', edgecolor='k', s=50
    )

    # Title and labels
    ax.set_title(f'MLP Decision Boundary (alpha={alpha})')
    ax.set_xlabel('Sepal Length (standardized)')
    ax.set_ylabel('Petal Length (standardized)')

    # Remove axes for clarity
    ax.set_xticks([])
    ax.set_yticks([])

    # Tight layout
    plt.tight_layout()

    # Save the plot to a BytesIO object
    buf = BytesIO()
    plt.savefig(buf, format='png')
    plt.close(fig)
    buf.seek(0)
    return Image.open(buf)


Train MLP with Varying Alpha Values and Collect Images

In [20]:
from sklearn.neural_network import MLPClassifier

def create_decision_boundary_gif(alpha_values, X_train, y_train, n_neurons):
    # List to store images
    images = []

    for idx, alpha in enumerate(alpha_values):
        print(f"Processing alpha={alpha:.4f} ({idx + 1}/{len(alpha_values)})")

        # Create and train the MLP
        mlp = MLPClassifier(hidden_layer_sizes=(n_neurons,), alpha=alpha, random_state=42)
        mlp.fit(X_train, y_train)

        # Plot decision boundary and get the image
        img = plot_decision_boundary(mlp, X_train, y_train, alpha)
        images.append(img)

    # Save the images as a GIF
    gif_filename = 'mlp_classification_boundaries.gif'
    images[0].save(
        gif_filename,
        save_all=True,
        append_images=images[1:],
        duration=500,
        loop=0
    )

    print(f"GIF saved as '{gif_filename}'")

    # Return the gif filename
    return gif_filename


## RUN

In [25]:
# Use np.logspace to generate alpha values, with at least 20 values
alpha_values = np.logspace(-3, 3, 20)

# Define the number of neurons in the hidden layer
n_neurons = 15

# Create the decision boundary GIF
gif_dir = create_decision_boundary_gif(alpha_values, X_train, y_train, n_neurons)


Processing alpha=0.0010 (1/20)
Processing alpha=0.0021 (2/20)
Processing alpha=0.0043 (3/20)
Processing alpha=0.0089 (4/20)
Processing alpha=0.0183 (5/20)
Processing alpha=0.0379 (6/20)
Processing alpha=0.0785 (7/20)
Processing alpha=0.1624 (8/20)
Processing alpha=0.3360 (9/20)
Processing alpha=0.6952 (10/20)
Processing alpha=1.4384 (11/20)
Processing alpha=2.9764 (12/20)
Processing alpha=6.1585 (13/20)
Processing alpha=12.7427 (14/20)
Processing alpha=26.3665 (15/20)
Processing alpha=54.5559 (16/20)
Processing alpha=112.8838 (17/20)
Processing alpha=233.5721 (18/20)
Processing alpha=483.2930 (19/20)
Processing alpha=1000.0000 (20/20)
GIF saved as 'mlp_classification_boundaries.gif'


Your gif should look like this:

<div style="text-align: center;">

### **Multilayer Perceptron Classification Boundaries**

![Classification Boundaries](mlp_classification_boundaries_example.gif)

*Figure 1: Demonstration of classification boundaries created by a Multilayer Perceptron (MLP) model.*

</div>

